In [9]:
!pip install pandas matplotlib seaborn phik imblearn


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from phik.report import plot_correlation_matrix
import numpy as np
from scipy.stats import shapiro

# загружаем класс pipeline
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import FeatureUnion

# загружаем классы для подготовки данных
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer

# загружаем класс для работы с пропусками
from sklearn.impute import SimpleImputer

# загружаем функцию для работы с метриками
from sklearn.metrics import f1_score, mean_absolute_error, classification_report

# импортируем класс RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

from imblearn.over_sampling import SMOTE

# загружаем нужные модели
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

import warnings

In [11]:
pd.set_option('display.float_format', '{:,.2f}'.format)
warnings.filterwarnings('ignore')

## Загрузка и подготовка данных

### Тренировочная выборка

In [12]:
df_train = pd.read_csv('datasets/kaggle_startups_train_01.csv')
df_train.head(10)

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,Online Reservations|Restaurants,"828,626.00",operating,CHE,25,Zurich,Zürich,2,2010-06-24,2011-05-01,2014-12-01,NaN
1,EarLens,Manufacturing|Medical|Medical Devices,"42,935,019.00",operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaN
2,Reviva Pharmaceuticals,Biotechnology,"35,456,381.00",operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaN
3,Sancilio and Company,Health Care,"22,250,000.00",operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaN
4,WireTough Cylinders,Manufacturing,NaN,operating,USA,VA,VA - Other,Bristol,1,2010-07-30,2012-02-01,2012-02-01,NaN
5,Connected Sports Ventures,Mobile,"4,300,000.00",operating,USA,NJ,Newark,Princeton,1,2011-04-21,2012-11-12,2012-11-12,NaN
6,Attensity,Analytics|Business Analytics|Social CRM|Social...,"90,000,000.00",operating,USA,CA,SF Bay Area,Redwood City,1,2000-01-01,2014-05-14,2014-05-14,NaN
7,Mesh Networks,Software,"4,300,000.00",operating,USA,TX,Houston,Houston,1,2005-01-01,2014-11-09,2014-11-09,NaN
8,AngioScore,Biotechnology,"42,000,000.00",operating,USA,CA,SF Bay Area,Fremont,2,2003-01-01,2007-10-09,2011-04-20,NaN
9,Vidatronic,Semiconductors,"1,250,500.00",operating,USA,TX,Austin,College Station,2,2010-01-01,2011-08-23,2013-03-21,NaN


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52879 entries, 0 to 52878
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               52878 non-null  object 
 1   category_list      50374 non-null  object 
 2   funding_total_usd  42753 non-null  float64
 3   status             52879 non-null  object 
 4   country_code       47351 non-null  object 
 5   state_code         46082 non-null  object 
 6   region             46489 non-null  object 
 7   city               46489 non-null  object 
 8   funding_rounds     52879 non-null  int64  
 9   founded_at         52879 non-null  object 
 10  first_funding_at   52858 non-null  object 
 11  last_funding_at    52879 non-null  object 
 12  closed_at          4962 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 5.2+ MB


Описание колонок:
 
- `name` — наименование компании;
- `category_list` — категории деятельности компании;
- `funding_total_usd` — сумма привлечённых инвестиций в долларах;
- `status` — статус компании;
- `country_code` — код страны;
- `state_code` — код штата;
- `region` — регион;
- `city` — город;
- `funding_rounds` — число раундов, в которых компания привлекала инвестиции;
- `founded_at` — дата основания фонда;
- `first_funding_at` — дата первого финансирования;
- `last_funding_at` — дата последнего финансирования;
- `closed_at` — дата закрытия;

Наименование колонок корректное, у некоторых колонок не соответствуют типы.


### Тестовая выборка

#### Входные признаки тестовой выборки

In [14]:
df_test_features = pd.read_csv('datasets/kaggle_startups_test_01.csv')
df_test_features.head(10)

,name,category_list,funding_total_usd,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Crystalsol,Clean Technology,"2,819,200.00",NIC,17,NaN,NaN,1,2008-06-01,2009-07-01,2009-07-01,NaN
1,JBI Fish & Wings,Hospitality,NaN,USA,TN,TN - Other,Humboldt,1,2010-07-25,2010-07-28,2010-07-28,NaN
2,COINPLUS,Finance,"428,257.00",LUX,3,Esch-sur-alzette,Esch-sur-alzette,2,2014-06-16,2014-05-15,2014-09-18,NaN
3,Imagine Communications,Software|Video|Video Streaming,"34,700,000.00",USA,CA,San Diego,San Diego,4,2005-01-01,2005-01-01,2010-04-20,NaN
4,DNA13,Software,"4,530,000.00",CAN,ON,Ottawa,Ottawa,1,2001-01-01,2007-05-08,2007-05-08,NaN
5,Quickfire Games,Design|Entertainment|Games,"160,000.00",NaN,NaN,NaN,NaN,2,2013-09-01,2013-09-18,2014-09-18,NaN
6,Sente Inc.,Biotechnology,"26,842,000.00",USA,CA,San Diego,Encinitas,5,2007-01-01,2009-01-31,2014-06-02,NaN
7,Triosyn,Health Care|Medical|Therapeutics,"4,000,000.00",NaN,NaN,NaN,NaN,1,2001-12-26,2003-06-02,2003-06-02,NaN
8,Urgent.ly,Software,"8,710,000.00",USA,VA,"Washington, D.C.",Sterling,3,2013-01-01,2014-04-17,2015-09-29,NaN
9,Map Decisions,Software,"13,200.00",USA,PA,Allentown,Bethlehem,1,2012-05-15,2013-08-09,2013-08-09,NaN


In [15]:
df_test_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13211 entries, 0 to 13210
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               13211 non-null  object 
 1   category_list      12610 non-null  object 
 2   funding_total_usd  10616 non-null  float64
 3   country_code       11827 non-null  object 
 4   state_code         11512 non-null  object 
 5   region             11618 non-null  object 
 6   city               11620 non-null  object 
 7   funding_rounds     13211 non-null  int64  
 8   founded_at         13211 non-null  object 
 9   first_funding_at   13211 non-null  object 
 10  last_funding_at    13211 non-null  object 
 11  closed_at          1234 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 1.2+ MB


Описание колонок:
 
- `name` — наименование компании;
- `category_list` — категории деятельности компании;
- `funding_total_usd` — сумма привлечённых инвестиций в долларах;
- `country_code` — код страны;
- `state_code` — код штата;
- `region` — регион;
- `city` — город;
- `funding_rounds` — число раундов, в которых компания привлекала инвестиции;
- `founded_at` — дата основания фонда;
- `first_funding_at` — дата первого финансирования;
- `last_funding_at` — дата последнего финансирования;
- `closed_at` — дата закрытия;

Наименование колонок корректное, у колонок с датами не соответствуют типы.

#### Целевой признак тестовой выборки

In [16]:
df_test_target = pd.read_csv('datasets/kaggle_startups_sample_submit_02.csv')
df_test_target.head(10)

,name,status
0,Crystalsol,operating
1,JBI Fish & Wings,operating
2,COINPLUS,operating
3,Imagine Communications,operating
4,DNA13,operating
5,Quickfire Games,operating
6,Sente Inc.,operating
7,Triosyn,operating
8,Urgent.ly,operating
9,Map Decisions,operating


In [17]:
df_test_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13211 entries, 0 to 13210
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    13211 non-null  object
 1   status  13211 non-null  object
dtypes: object(2)
memory usage: 206.5+ KB


Описание колонок:
 
- `name` — наименование компании;
- `status` — статус компании;

Наименование колонок корректное, все типы верные.

#### Итоговая тестовая выборка

In [18]:
df_test = pd.merge(df_test_features, df_test_target, on='name')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13211 entries, 0 to 13210
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               13211 non-null  object 
 1   category_list      12610 non-null  object 
 2   funding_total_usd  10616 non-null  float64
 3   country_code       11827 non-null  object 
 4   state_code         11512 non-null  object 
 5   region             11618 non-null  object 
 6   city               11620 non-null  object 
 7   funding_rounds     13211 non-null  int64  
 8   founded_at         13211 non-null  object 
 9   first_funding_at   13211 non-null  object 
 10  last_funding_at    13211 non-null  object 
 11  closed_at          1234 non-null   object 
 12  status             13211 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.3+ MB


Описание колонок:
 
- `name` — наименование компании;
- `category_list` — категории деятельности компании;
- `funding_total_usd` — сумма привлечённых инвестиций в долларах;
- `country_code` — код страны;
- `state_code` — код штата;
- `region` — регион;
- `city` — город;
- `funding_rounds` — число раундов, в которых компания привлекала инвестиции;
- `founded_at` — дата основания фонда;
- `first_funding_at` — дата первого финансирования;
- `last_funding_at` — дата последнего финансирования;
- `closed_at` — дата закрытия;
- `status` — статус компании;

Наименование колонок корректное, у колонок с датами не соответствуют типы. После обледенения таблиц все данные на месте.

## Предобработка данных

Вспомогательные функции

In [19]:
# Определим имена колонок для отображения
column_names = {
    "name": "наименование компании",
    "category_list": "категории деятельности компании",
    "funding_total_usd": "сумма привлечённых инвестиций в долларах",
    "country_code": "код страны",
    "state_code": "код штата",
    "region": "регион",
    "city": "город",
    "funding_rounds": "число раундов, в которых компания привлекала инвестиции",
    "founded_at": "дата основания фонда",
    "first_funding_at": "дата первого финансирования",
    "last_funding_at": "дата последнего финансирования",
    "closed_at": "дата закрытия",
    "status": "статус компании"
}

In [20]:
# Функция для обработки пропусков
def isna_rate(df):
    df_isna = (pd.DataFrame(round(df.isna().mean() * 100, 2)).sort_values(by=0, ascending=False))
    df_isna.columns = ['Доля_пропусков']
    df_isna['Кол-во_пропусков'] = df.isna().sum()
    df_isna['Всего_записей'] = df.shape[0]
    display(df_isna.query('Доля_пропусков > 0'))

In [21]:
# Функция для обработки строковых колонок
def check_unique(df):
    for column in df.select_dtypes(include='object').columns.tolist():
        print(f"Колонка \"{column}\":")
        print(df[column].value_counts(), "\n")

In [22]:
# Проверяет на дубликаты
def check_duplicated(df):
    dupl = df.duplicated().sum()
    print("Количество дубликатов:", dupl)
    if dupl > 0:
        display(df[df.duplicated()])
    else:
        print("Дубликаты отсутствуют")

### Тренировочная выборка

#### Типы

Приведем типы с датами в верный формат

In [23]:
df_train['founded_at'] = pd.to_datetime(df_train['founded_at'])
df_train['first_funding_at'] = pd.to_datetime(df_train['first_funding_at'])
df_train['last_funding_at'] = pd.to_datetime(df_train['last_funding_at'])
df_train['closed_at'] = pd.to_datetime(df_train['closed_at'])

Приведем типы со статусом к 0 / 1

In [24]:
# Функция для подсчета количества категорий
def status_int(row):
    if row == 'closed':
        return 1
    else:
        return 0


# Создадим новую колонку кол-во категорий
df_train['status'] = df_train['status'].apply(status_int)

In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52879 entries, 0 to 52878
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               52878 non-null  object        
 1   category_list      50374 non-null  object        
 2   funding_total_usd  42753 non-null  float64       
 3   status             52879 non-null  int64         
 4   country_code       47351 non-null  object        
 5   state_code         46082 non-null  object        
 6   region             46489 non-null  object        
 7   city               46489 non-null  object        
 8   funding_rounds     52879 non-null  int64         
 9   founded_at         52879 non-null  datetime64[ns]
 10  first_funding_at   52858 non-null  datetime64[ns]
 11  last_funding_at    52879 non-null  datetime64[ns]
 12  closed_at          4962 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(1), int64(2), object(6)
memory 

Приведем типы список категорий

In [51]:
# Разделяем категории в колонке category_list по символу '|' и создаем новую колонку
df_train['categories'] = df_train['category_list'].str.split('|')



# # Создаем новый DataFrame, в котором каждая категория будет отдельной строкой
categories_df = df_train['category_list'].str.split('|').explode('categories')


# Считаем количество вхождений каждой категории
category_counts = categories_df.value_counts()


# Получаем топ 200 категорий
top_200_categories = category_counts.head(200).index.tolist()

category_counts
# # Функция для проверки наличия категорий из топа в списке категорий
# def check_top_categories(category_list):
#     if any(category in top_200_categories for category in category_list):
#         return '|'.join(sorted(category_list))
#     else:
#         return 'Other'

# top_200_categories
# # Применяем функцию к колонке categories и создаем новую колонку с обновленными категориями
# df['updated_categories'] = df['categories'].apply(check_top_categories)
# 
# # Сохраняем обновленный DataFrame в файл или используем далее по необходимости
# df.to_csv('updated_file.csv', index=False)

category_list
Software               7033
Mobile                 4405
Biotechnology          3683
E-Commerce             3316
Curated Web            2417
                       ... 
Made in Italy             1
Private Corrections       1
Direct Advertising        1
Labor Optimization        1
Elderly                   1
Name: count, Length: 854, dtype: int64

#### Пропуски

In [ ]:
isna_rate(df_train)

In [ ]:
# Удалим значения с пустыми датами первого финансирования так как их 0,04%
df_train = df_train.query('~first_funding_at.isna()')

In [ ]:
# Проверим данные с не заполнеными данными страна, категория, финансирование
df_train.query('country_code.isna() & category_list.isna() & funding_total_usd.isna()')

Так как такие данные содержат пустые поля во всех ключевых параметров уберем их

In [ ]:
df_train = df_train.query('~(country_code.isna() & category_list.isna() & funding_total_usd.isna())')
isna_rate(df_train)

In [ ]:
# Проверим данные с заполнеными страной и не заполненой категорией 
df_train.query('~country_code.isna() & category_list.isna()')

Есть пропуски в нескольких колонках, обработаем им далее в пайплайне.

#### Неявные дубликаты

In [ ]:
# check_unique(df_train)
ss()

#### Явные дубликаты

In [ ]:
# check_duplicated(df_train)

### Тестовая выборка

#### Типы

Приведем типы с датами в верный формат

In [ ]:
df_test['founded_at'] = pd.to_datetime(df_train['founded_at'])
df_test['first_funding_at'] = pd.to_datetime(df_train['first_funding_at'])
df_test['last_funding_at'] = pd.to_datetime(df_train['last_funding_at'])
df_test['closed_at'] = pd.to_datetime(df_train['closed_at'])

Приведем типы со статусом к 0 / 1

In [ ]:
df_test['status'] = df_test['status'].apply(status_int)

In [ ]:
df_test.info()

#### Пропуски

In [ ]:
# isna_rate(df_test)

Есть пропуски в нескольких колонках, обработаем им далее в пайплайне.

#### Неявные дубликаты

In [ ]:
# check_unique(df_train)

#### Явные дубликаты

In [ ]:
# check_duplicated(df_test)

## Исследовательский анализ данных

In [ ]:
# Графики для анализа числовых колонок
def column_quantitative_info(df, column):
    desc = df[column].describe()
    print(desc)
    if df[column].dtype == 'int64':
        n_bins = (df[column].max() - df[column].min()) + 1
        if n_bins > 100:
            binwidth = None
            discrete = False
            bins = 'auto'
        else:
            bins = n_bins
            discrete = True
            binwidth = 1
    else:
        binwidth = None
        discrete = False
        bins = 'auto'
    print(discrete, bins)
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
    sns.histplot(data=df[column], kde=False, ax=axes[0], discrete=discrete, binwidth=binwidth, bins=bins)
    axes[0].axvline(df[column].mean(), color='red', linestyle='--', label='Mean')
    axes[0].set_title('Гистограмма распределения')
    axes[0].set_ylabel('Частота')
    axes[0].set_xlabel(column_names[column])
    sns.boxplot(data=df[column], palette='rainbow', ax=axes[1])
    axes[1].set_title('Разброс значений признаков')
    axes[1].set_ylabel('Значение')
    stats.probplot(df[column], plot=sns.mpl.pyplot, dist="norm")
    axes[2].set_title('Q-Q график')
    axes[2].set_ylabel('Квантили эмпирических данных')
    axes[2].set_xlabel('Теоретические квантили')
    plt.show()

In [ ]:
# Числовые колонки
def quantitative_info(df):
    for column in df.loc[:, df.columns != 'id'].select_dtypes(include='number').columns.tolist():
        stat, p_val = shapiro(df[column])
        print(f'Колонка: \'{column}\':')
        if p_val < 0.5:
            print('Распределение: не нормальное')
        else:
            print('Распределение: нормальное')
        column_quantitative_info(df, column)

In [ ]:
# Строковые колонки   
def categorical_info(df):
    for column in df.loc[:, df.columns != 'id'].select_dtypes(include='object').columns.tolist():
        print(f'Колонка: \'{column_names[column]}\':')
        print((df[column].describe()))
        df[column].value_counts().plot(title=column_names[column], autopct='%1.1f%%', kind='pie', figsize=(7, 7))
        plt.ylabel(f"Соотношение {column_names[column]}")
        plt.show()

### Тренировочная выборка

In [ ]:
RANDOM_STATE = 42
# Сделаем sample_df для анализа
df_train_sample = df_train.sample(n=1000, random_state=RANDOM_STATE).reset_index(drop=True)

#### Количественные колонки

In [ ]:
# quantitative_info(df_train_sample)

Признаки имеют нормальное и не нормальное распределение.

Аномальных значений как таковых не выявлено.

#### Категориальные колонки

In [ ]:
# categorical_info(df_train_sample.drop(['name'], axis=1))

Выводы:
- Популярная категория Software.
- Большинство компаний из США.
- Большинство компаний из города San Francisco.
- Большинство компаний из региона SF Bay Area.
- Большинство компаний не закрыты.

### Тестовая выборка

In [ ]:
# Сделаем sample_df для анализа
df_test_sample = df_test.sample(n=1000, random_state=RANDOM_STATE).reset_index(drop=True)

#### Количественные колонки

In [ ]:
# quantitative_info(df_test_sample)

Признаки имеют нормальное и не нормальное распределение.

Аномальных значений как таковых не выявлено.

#### Категориальные колонки

In [ ]:
# categorical_info(df_test_sample.drop(['name'], axis=1))

Выводы:
- Популярная категория Software.
- Большинство компаний из США.
- Большинство компаний из города San Francisco.
- Большинство компаний из региона SF Bay Area.
- Статусы компании распределены почти равномерно.

## Подготовка данных

In [ ]:
def show_phik_matrix(df):
    phik_overview = df.sample(1000, random_state=RANDOM_STATE).phik_matrix()
    # визуализация тепловой карты коэффициентов корреляции
    plot_correlation_matrix(
        phik_overview.values,
        x_labels=phik_overview.columns,
        y_labels=phik_overview.index,
        title=r"correlation $\phi_K$",
        fontsize_factor=1.5,
        figsize=(15, 12)
    )

### Матрица корреляции на стандартных данных

Изучим коэффициенты корреляции между всеми признаками с целью определения наличия мультиколлинеарности. Для признаков с нормальным распределением будем применять коэффициент Пирсона, а для остальных - коэффициент Спирмена.

In [ ]:
show_phik_matrix(df_train)

Наблюдается утечка целевого признака, по параметрам с датами закрытия

### Дополнительные колонки

#### Заполним колонки дата закрытия

In [ ]:
df_train_old = df_train
# Заполним пустые значения в колонке дата закрытия на текущую дату что бы посчитать время существование компании.
df_train.closed_at.fillna(pd.to_datetime('2018-01-01'), inplace=True)
df_train.info()

In [ ]:
df_test_old = df_test

# Заполним пустые значения в колонке дата закрытия на текущую дату что бы посчитать время существование компании.
df_test.closed_at = pd.to_datetime('2018-01-01')
df_test.info()

#### Дополнительные колонки из колонок с датами

In [ ]:
# def create_new_columns(df):
#     # Возраст компании на момент первого финансирования (first_funding_age):
#     df['age_first_funding'] = (df['first_funding_at'] - df['founded_at']).dt.days
#     
#     return df

In [ ]:
df_train['count_usd_day_life'] = df_train.funding_total_usd / (df_train.closed_at - df_train.founded_at).dt.days

In [ ]:
df_train = df_train.query('closed_at >= founded_at')

In [ ]:
show_phik_matrix(df_train.drop([
    'first_funding_at',
    'last_funding_at',
    'founded_at',
    'closed_at',
    'name'
], axis=1))

In [ ]:
# create_new_columns(df_test)
ф()

#### Дополнительные колонки из колонки с категориями

In [ ]:
unique_categories = df_train['category_list'].dropna().str.split('|').explode().unique()


def category_list_to_binary_columns(df, column_name='category_list'):
    for category in unique_categories:
        df[f'cat_{category}'] = df[column_name].str.contains(category).fillna(0).astype(int)


In [ ]:
# Разделяем строку на список категорий и создаем бинарные колонки для обучающей
# category_list_to_binary_columns(df_train)
df_train.head()

In [ ]:
# Разделяем строку на список категорий и создаем бинарные колонки для обучающей
# category_list_to_binary_columns(df_test)
df_test.head()

In [ ]:
# df_train['category_list'].dropna().str.split('|').explode().unique()

In [ ]:
df_train_sample = df_train
X = df_train_sample.drop([
    'name',
    'first_funding_at',
    'last_funding_at',
    'founded_at',
    'closed_at',
    'category_list'
], axis=1)
y = df_train_sample['status']
show_phik_matrix(X)

### Пайплайн подготовки данных

In [27]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52879 entries, 0 to 52878
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               52878 non-null  object        
 1   category_list      50374 non-null  object        
 2   funding_total_usd  42753 non-null  float64       
 3   status             52879 non-null  int64         
 4   country_code       47351 non-null  object        
 5   state_code         46082 non-null  object        
 6   region             46489 non-null  object        
 7   city               46489 non-null  object        
 8   funding_rounds     52879 non-null  int64         
 9   founded_at         52879 non-null  datetime64[ns]
 10  first_funding_at   52858 non-null  datetime64[ns]
 11  last_funding_at    52879 non-null  datetime64[ns]
 12  closed_at          4962 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(1), int64(2), object(6)
memory 

In [28]:
# создаём пайплайн для подготовки признаков из списка ohe_columns: заполнение пропусков и OHE-кодирование
# SimpleImputer + OHE
ohe_pipe = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='most_frequent'),
    OneHotEncoder(drop='first', handle_unknown='ignore')
)

num_pipe = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='median'),
    StandardScaler()
)

# создаём общий пайплайн для подготовки данных
preprocessor = make_column_transformer(
    (ohe_pipe, make_column_selector(dtype_exclude='number')),
    (num_pipe, make_column_selector(dtype_include='number')),
    remainder='passthrough'
)

pipe_preparation = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('resample', SMOTE(random_state=RANDOM_STATE))
    ]
)

pipe_preparation

NameError: name 'RANDOM_STATE' is not defined

## Обучение модели

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X.drop(['status'], axis=1), y, test_size=0.25, random_state=RANDOM_STATE)
print(f'X {len(X_train)}/{len(X_test)}')
print(f'y {len(y_train)}/{len(y_test)}')

NameError: name 'X' is not defined

In [30]:
pipe_final = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('resample', SMOTE(random_state=RANDOM_STATE)),
        ('models', LogisticRegression(random_state=RANDOM_STATE))
    ]
)
pipe_final

NameError: name 'RANDOM_STATE' is not defined

In [31]:
pipe_final.fit(X_train, y_train)
preds = pipe_final.predict(X_test)
print(classification_report(y_test, preds))

NameError: name 'pipe_final' is not defined

In [32]:
preds = pipe_final.predict(df_test.drop(['status'], axis=1))
print(classification_report(df_test['status'], preds))

NameError: name 'pipe_final' is not defined

In [33]:
# Настройка гиперпараметров моделей
param_grid = [
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced')],
        'models__C': range(1, 5),
        'models__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'models__penalty': ['l1', 'l2'],
        'preprocessor__pipeline-2__standardscaler': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },

    # словарь для модели DecisionTreeClassifier() 
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
        'models__criterion': ['log_loss', 'entropy', 'gini'],
        'models__max_depth': range(1, 100),
        'models__min_samples_split': range(2, 10),
        'models__min_samples_leaf': range(1, 10),
        'models__max_features': range(1, 10),
        'preprocessor__pipeline-2__standardscaler': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    # словарь для модели RandomForestClassifier() 
    {
        'models': [RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
        'models__n_estimators': range(1, 100),
        'models__max_depth': range(1, 100),
        'preprocessor__pipeline-2__standardscaler': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    # словарь для модели GradientBoostingClassifier()
    {
        'models': [GradientBoostingClassifier(random_state=RANDOM_STATE)],
        'models__n_estimators': range(1, 100),
        'models__max_depth': range(1, 100),
        'preprocessor__pipeline-2__standardscaler': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    # словарь для модели MLP()
    {
        'models': [MLPClassifier(random_state=RANDOM_STATE)],
        'models__hidden_layer_sizesr': [(64,), (128,), (64, 64)],
        'models__hidden_layer_alpha': [(64,), (128,), (64, 64)],
        'preprocessor__pipeline-2__standardscaler': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]

randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='neg_mean_absolute_error'
)

randomized_search

NameError: name 'RANDOM_STATE' is not defined

In [34]:
randomized_search.fit(X_train, y_train)
print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print('Метрика лучшей модели на тренировочной выборке:', round(randomized_search.best_score_, 2))

NameError: name 'randomized_search' is not defined

In [35]:
randomized_search.best_estimator_.named_steps['models']

NameError: name 'randomized_search' is not defined

In [36]:
preds = randomized_search.predict(X_test)
print(classification_report(y_test, preds))

NameError: name 'randomized_search' is not defined

In [37]:
aa()
pipe_final.named_steps['models'].feature_importances_

NameError: name 'aa' is not defined

In [38]:
feature_importances = pipe_final.named_steps['models'].feature_importances_
feature_names = preprocessor.get_feature_names_out()
feature_names

NameError: name 'pipe_final' is not defined

In [39]:
pd.DataFrame(feature_importances, index=feature_names).head(10).sort_values(by=0).plot(kind='barh')

NameError: name 'feature_importances' is not defined

In [40]:
# model = randomized_search.best_estimator_.named_steps['models']
# preprocessor = randomized_search.best_estimator_.named_steps['preprocessor']
# 
# feature_importances = pd.DataFrame(
#     {
#         'Feature': preprocessor.get_feature_names_out(),
#         'Importance': model.feature_importances_
#     }).sort_values(by='Importance', ascending=False)
# feature_importances.head(10)

In [41]:
# ax = sns.barplot(x='Importance', y='Feature', data=feature_importances.head(10))
# ax.set_title('Важность признаков', fontsize=16)
# ax.set_xlabel('Важность (доля)', fontsize=14)
# ax.set_ylabel('Признак', fontsize=14)
# plt.show()